# FPL Squad Optimizer

This Jupyter Notebook uses a knapsack algorithm to create an Fantasy Premier League squad of 15 players optimized around a specified metric (goals, clean sheets, points etc.)

## Import Libraries

In [2]:
import requests
import pandas as pd
import numpy as np
import itertools
import copy

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Fetch data from the FPL API and clean it

In [3]:
# FPL API URL
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
response = requests.get(url)
json = response.json()

# JSON keys
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [4]:
# storing json outputs as dataframes

elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])

In [5]:
# Pulling in player position into slim_elements_df

elements_df['position'] = elements_df.element_type.map(elements_types_df.set_index('id').singular_name)
elements_df['team_name'] = elements_df.team.map(teams_df.set_index('id').name)

In [6]:
# Filtering out only the necessary columns

slim_elements_df = elements_df[['first_name', 'second_name','team_name','position','news','selected_by_percent','in_dreamteam',
                                'now_cost','form','points_per_game','minutes','goals_scored','assists','clean_sheets',
                                'goals_conceded','clean_sheets','goals_conceded','yellow_cards','red_cards','saves','bonus',
                                'transfers_in','value_season','total_points','influence','creativity','threat','ict_index']]

# numeric columns:

numeric_cols = ['selected_by_percent','form','points_per_game','value_season','influence','creativity','threat','ict_index']

In [7]:
# convering columns into numeric data type

for col in numeric_cols:
    slim_elements_df[col] = pd.to_numeric(slim_elements_df[col])


In [8]:
# actual cost of the player is now_cost/10
slim_elements_df['actual_cost'] = slim_elements_df['now_cost']/10

# creating additional metrics
slim_elements_df['games_completed'] = slim_elements_df['minutes']/90
slim_elements_df['points_per_90_mins'] = slim_elements_df['total_points']/slim_elements_df['games_completed']
slim_elements_df['ga_per_90_mins'] = (slim_elements_df['goals_scored']+slim_elements_df['assists'])/slim_elements_df['games_completed']
slim_elements_df['points_per_million'] = slim_elements_df['total_points']/slim_elements_df['actual_cost']

# eligible players
eligible_players = slim_elements_df[slim_elements_df['news'] == '']

# create a dataframe with only differentials: owned by less than 20%
differentials = slim_elements_df.loc[(slim_elements_df['news'] == '') & (slim_elements_df['selected_by_percent'] <= 20)]

## Python functions for the knapsack algorithm

In [14]:
def knapsack_solution(players, player_costs, player_values, max_cost, count):
    
    """
    function that returns the knapsack cost matrix
    """
  
    num_players = len(players)
  
    cost_matrix = [[[0 for k in range(count+1)] for j in range(max_cost+1)] for i in range(num_players)]
    
    for i in range(num_players):
        for j in range(max_cost+1):
            for k in range(count+1):
                if (player_costs[i] > j) or (1 > k):
                    cost_matrix[i][j][k] = cost_matrix[i-1][j][k]
                else: 
                    cost_matrix[i][j][k] = max(cost_matrix[i-1][j][k], player_values[i]+cost_matrix[i-1][j-player_costs[i]][k-1])

    return cost_matrix
    

In [15]:
def get_used_items(players, player_costs, player_values, max_cost, count, cost_matrix):
    
    """
    function that returns the used players from the cost matrix
    """
    
    playerIndex = len(players) - 1
    
    currentCost = -1
    currentCount = count
    marked = [0 for k in range(len(players))]

    bestValue = -1
    
    for j in range(max_cost+1):
        value = cost_matrix[playerIndex][j][count]
        if (bestValue == -1) or (value > bestValue):
            currentCost = j
            bestValue = value
    
    while (playerIndex >= 0 and currentCost >= 0 and currentCount >= 0):
        if (playerIndex == 0 and cost_matrix[playerIndex][currentCost][currentCount] > 0) or (cost_matrix[playerIndex][currentCost][currentCount] != cost_matrix[playerIndex-1][currentCost][currentCount]):
            marked[playerIndex] = 1
            currentCost = currentCost - player_costs[playerIndex]
            currentCount = currentCount - 1
        playerIndex = playerIndex - 1

    return marked
      

## Python functions to optimize keepers, defenders, midfielders, forwards

The knapsack algorithm will return an optimal squad of 15 players, but will not ensure that players are distributed into 2 goalkeepers, 5 defenders, 5 midfielders and 3 forwards.

For this, we will follow these steps:
1. Get every combination of 4 numbers adding up to 100 (for the total costs of goalkeepers, defence, midfield, attack)
2. For each of these combinations, run the knapsack algorithm individually for each part of the squad
3. Choose the combination that gets the highest value of the target metric to be optimized

In [16]:
def optimum_keepers(eligible_players, maximum_cost, opt_metric):
    
    max_cost = maximum_cost * 10
    
    gk_df = eligible_players[eligible_players['position'] == 'Goalkeeper']
    gk_df = gk_df.reset_index()
    goalkeepers = gk_df.index.tolist()
    goalkeeper_costs = (gk_df['now_cost']).tolist()
    goalkeeper_values = gk_df[opt_metric].tolist()
    
    cost_matrix = knapsack_solution(goalkeepers, goalkeeper_costs, goalkeeper_values, max_cost, 2)
    
    used_players = get_used_items(goalkeepers, goalkeeper_costs, goalkeeper_values, max_cost, 2, cost_matrix)
    
    player_indices = []
    
    for i in range(len(used_players)):
        if used_players[i] == 1:
            player_indices.append(i)
        
    players = pd.DataFrame()
    
    for index in range(len(player_indices)):
        players = pd.concat([players, gk_df.iloc[[player_indices[index]]]])
        
    final = players[['first_name', 'second_name', 'team_name', 'position', 'selected_by_percent', 'actual_cost', 'total_points', opt_metric]]
    
    return final.loc[:,~final.columns.duplicated()].copy()

In [17]:
def optimum_defence(eligible_players, maximum_cost, opt_metric):
    
    max_cost = maximum_cost * 10
    
    def_df = eligible_players[eligible_players['position'] == 'Defender']
    def_df = def_df.reset_index()
    defenders = def_df.index.tolist()
    defender_costs = (def_df['now_cost']).tolist()
    defender_values = def_df[opt_metric].tolist()
    
    cost_matrix = knapsack_solution(defenders, defender_costs, defender_values, max_cost, 5)
    
    used_players = get_used_items(defenders, defender_costs, defender_values, max_cost, 5, cost_matrix)
    
    player_indices = []
    
    for i in range(len(used_players)):
        if used_players[i] == 1:
            player_indices.append(i)
        
    players = pd.DataFrame()
    
    for index in range(len(player_indices)):
        players = pd.concat([players, def_df.iloc[[player_indices[index]]]])
        
    final = players[['first_name', 'second_name', 'team_name', 'position', 'selected_by_percent', 'actual_cost', 'total_points', opt_metric]]
    
    return final.loc[:,~final.columns.duplicated()].copy()

In [18]:
def optimum_midfield(eligible_players, maximum_cost, opt_metric):
    
    max_cost = maximum_cost * 10
    
    mid_df = eligible_players[eligible_players['position'] == 'Midfielder']
    mid_df = mid_df.reset_index()
    midfielders = mid_df.index.tolist()
    midfielder_costs = (mid_df['now_cost']).tolist()
    midfielder_values = mid_df[opt_metric].tolist()
    
    cost_matrix = knapsack_solution(midfielders, midfielder_costs, midfielder_values, max_cost, 5)
    
    used_players = get_used_items(midfielders, midfielder_costs, midfielder_values, max_cost, 5, cost_matrix)
    
    player_indices = []
    
    for i in range(len(used_players)):
        if used_players[i] == 1:
            player_indices.append(i)
        
    players = pd.DataFrame()
    
    for index in range(len(player_indices)):
        players = pd.concat([players, mid_df.iloc[[player_indices[index]]]])
        
    final = players[['first_name', 'second_name', 'team_name', 'position', 'selected_by_percent', 'actual_cost', 'total_points', opt_metric]]
    
    return final.loc[:,~final.columns.duplicated()].copy()

In [19]:
def optimum_attack(eligible_players, maximum_cost, opt_metric):
    
    max_cost = maximum_cost * 10
    
    att_df = eligible_players[eligible_players['position'] == 'Forward']
    att_df = att_df.reset_index()
    attackers = att_df.index.tolist()
    attacker_costs = (att_df['now_cost']).tolist()
    attacker_values = att_df[opt_metric].tolist()
    
    cost_matrix = knapsack_solution(attackers, attacker_costs, attacker_values, max_cost, 3)
    
    used_players = get_used_items(attackers, attacker_costs, attacker_values, max_cost, 3, cost_matrix)
    
    player_indices = []
    
    for i in range(len(used_players)):
        if used_players[i] == 1:
            player_indices.append(i)
        
    players = pd.DataFrame()
    
    for index in range(len(player_indices)):
        players = pd.concat([players, att_df.iloc[[player_indices[index]]]])
        
    final = players[['first_name', 'second_name', 'team_name', 'position', 'selected_by_percent', 'actual_cost', 'total_points', opt_metric]]
    
    return final.loc[:,~final.columns.duplicated()].copy()

# Cost Breakdowns

The following functions provide all the combination sums that add up to 100.

To avoid longer runtimes for the optimization functions, the costs for each section has a minimum threshold:
1. Keepers: 8 MM
2. Defence: 25 MM
3. Midfield: 30 MM
4. Attack: 20 MM

In [20]:
# Functions to get all the sum combinations

def print_all_sum_rec(target, current_sum, start, output, result):
    if current_sum == target:
        output.append(copy.copy(result))

    for i in range(start, target):
        temp_sum = current_sum + i
        if temp_sum <= target:
            result.append(i)
            print_all_sum_rec(target, temp_sum, i, output, result)
            result.pop()
        else:
            return

def print_all_sum(target):
    output = []
    result = []
    print_all_sum_rec(target, 0, 4, output, result)
    return output


In [35]:
# Function that selects only the combinations with 4 numbers

def cost_breakdown(number):
    breakdown = print_all_sum(number)
    combinations = []
    for i in breakdown:
        if len(i) == 4:
            if (i[0] >= 8) and (i[1] >= 25) and (i[2] >= 30) and (i[3] >= 20):
                combinations.append(i)
    return combinations

In [25]:
# Final optimization function

def optimize(opt_metric):
    costs_combinations = cost_breakdown(100)

    comb_df = pd.DataFrame(columns = ['costs', 'total_cost', opt_metric])
    
    for costs in costs_combinations:
        
        gk = optimum_keepers(eligible_players, costs[0], opt_metric)
        dfnc = optimum_defence(eligible_players, costs[1], opt_metric)
        mid = optimum_midfield(eligible_players, costs[2], opt_metric)
        att = optimum_attack(eligible_players, costs[3], opt_metric)
        
        final = pd.concat([gk, dfnc, mid, att])
        total_cost = final['actual_cost'].sum()
        optimised_metric = final[opt_metric].sum()
        cost_details = [costs, total_cost, optimised_metric]
        
        comb_df.loc[len(comb_df)] = cost_details

    return comb_df

In [36]:
opt_metric = 'total_points'

combs = optimize(opt_metric)
combs[opt_metric] = pd.to_numeric(combs[opt_metric])
combs.sort_values(by=[opt_metric], ascending=False).head(20)

,costs,total_cost,total_points
38,"[10, 26, 32, 32]",98.4,1120
49,"[11, 26, 31, 32]",97.9,1118
40,"[10, 27, 31, 32]",97.2,1117
37,"[10, 26, 31, 33]",97.2,1117
50,"[11, 27, 30, 32]",97.4,1116
41,"[10, 28, 30, 32]",98.9,1116
56,"[12, 26, 30, 32]",97.4,1116
48,"[11, 26, 30, 33]",97.4,1116
39,"[10, 27, 30, 33]",96.7,1115
36,"[10, 26, 30, 34]",96.7,1115


In [22]:
optimum_keepers(eligible_players, 10, opt_metric)
optimum_defence(eligible_players, 28, opt_metric)
optimum_midfield(eligible_players, 30, opt_metric)
optimum_attack(eligible_players, 32, opt_metric)

,first_name,second_name,team_name,position,selected_by_percent,actual_cost,total_points
38,Nick,Pope,Newcastle,Goalkeeper,21.0,5.3,62
40,Dean,Henderson,Nott'm Forest,Goalkeeper,12.6,4.7,61


,first_name,second_name,team_name,position,selected_by_percent,actual_cost,total_points
3,Benjamin,White,Arsenal,Defender,8.4,4.6,54
5,William,Saliba,Arsenal,Defender,32.7,5.2,61
74,Timothy,Castagne,Leicester,Defender,6.8,4.7,59
99,João,Cancelo,Man City,Defender,56.7,7.4,69
115,Kieran,Trippier,Newcastle,Defender,63.3,5.9,77


,first_name,second_name,team_name,position,selected_by_percent,actual_cost,total_points
6,Gabriel,Martinelli Silva,Arsenal,Midfielder,49.5,6.8,69
45,Pascal,Groß,Brighton,Midfielder,13.8,5.6,68
47,Leandro,Trossard,Brighton,Midfielder,26.2,7.0,78
91,Andreas,Hoelgebaum Pereira,Fulham,Midfielder,24.4,4.6,59
150,Miguel,Almirón Rejala,Newcastle,Midfielder,28.7,5.7,79


,first_name,second_name,team_name,position,selected_by_percent,actual_cost,total_points
7,Ivan,Toney,Brentford,Forward,15.3,7.4,70
31,Erling,Haaland,Man City,Forward,80.0,12.1,122
43,Harry,Kane,Spurs,Forward,25.4,11.6,83
